In [1]:
import json
import etri
import sys
from datetime import datetime
import time
from koreanframenet import kfn
from optparse import OptionParser

In [2]:
import targetid
import frameid
import argid

In [3]:
# options

now = datetime.now()
resultfname = './tmp/'+str(now.year)+'.'+str(now.month)+'.'+str(now.day)+'.'+str(time.time())

optpr = OptionParser()
optpr.add_option("--mode", dest='mode', type='choice', choices=['parsing', 'test'], default='parsing')
optpr.add_option("--targetid", dest='targetid', type='choice', choices=['baseline'], default='baseline')
optpr.add_option("--frameid", dest='frameid', type='choice', choices=['baseline'], default='baseline')
optpr.add_option("--argid", dest='argid', type='choice', choices=['baseline'], default='baseline')
optpr.add_option("--result", dest='resultfile', help="Saved result file", metavar="FILE", default=resultfname)
optpr.add_option("--input", dest='input', help="input a sentence", type="string", default=resultfname)
# (options, args) = optpr.parse_args()

optpr.mode = 'parsing'
optpr.targetid = 'baseline'
optpr.frameid = 'frequent'
optpr.argid = 'rulebased'

#print options
#sys.stderr.write("\nCOMMAND: "+' '.join(sys.argv) + '\n')
sys.stderr.write("\nPARSER SETTINGS\n_____________________\n")
sys.stderr.write("Target Identification     \t" + str(optpr.targetid) + '\n')
sys.stderr.write("Frame Identification      \t" + str(optpr.frameid) + '\n')
sys.stderr.write("Argument Identification   \t" + str(optpr.argid) + '\n')
if optpr.mode in ['parsing']:
    sys.stderr.write("RESULT WILL BE SAVED TO\t%s\n" %resultfname)


PARSER SETTINGS
_____________________
Target Identification     	baseline
Frame Identification      	frequent
Argument Identification   	rulebased
RESULT WILL BE SAVED TO	./tmp/2018.7.10.1531197262.7673354


In [4]:
# parset options

target_identifier = targetid.target_identifier
frame_identifier = frameid.frame_identifier
arg_identifier = argid.arg_identifier

In [16]:
def frameparsing(sent):
    conll = etri.getETRI_CoNLL2009(sent)
    conll_target = target_identifier(conll, optpr.targetid)
    conll_frame = frame_identifier(conll_target, optpr.frameid)
    conll_arg = arg_identifier(conll_frame, optpr.argid)
    
    return conll_arg

In [25]:
# parsing

def test():
    if optpr.mode == 'parsing':
        sent = optpr.input = "기계 학습(機械學習) 또는 머신 러닝(영어: machine learning)은 인공 지능의 한 분야로, 기계가 정보를 학습하도록 하는 알고리즘과 기술을 개발하는 분야를 말한다."
        parsed = frameparsing(sent)
        print(parsed)
        for i in parsed:
            for j in i:
                print(j)
            print('')
test()

[[0, '기계', '기계/NNG', '기계', 'NNG', 'NNG', '_', '_', 1, 1, 'NP', 'NP'], [1, '학습(機械學習)', '학습/NNG+(/SS+機械學習/SH+)/SS', '학습(機械學習)', 'NNG+SS+SH+SS', 'NNG+SS+SH+SS', '_', '_', 6, 6, 'NP_CNJ', 'NP_CNJ'], [2, '또는', '또는/MAJ', '또는', 'MAJ', 'MAJ', '_', '_', 6, 6, 'AP', 'AP'], [3, '머신', '머신/NNG', '머신', 'NNG', 'NNG', '_', '_', 4, 4, 'NP', 'NP'], [4, '러닝(영어:', '러닝/NNG+(/SS+영어/NNG+:/SP', '러닝(영어:', 'NNG+SS+NNG+SP', 'NNG+SS+NNG+SP', '_', '_', 6, 6, 'NP', 'NP'], [5, 'machine', 'machine/SL', 'machine', 'SL', 'SL', '_', '_', 6, 6, 'NP', 'NP'], [6, 'learning)은', 'learning/SL+)/SS+은/JX', 'learning)은', 'SL+SS+JX', 'SL+SS+JX', '_', '_', 10, 10, 'NP_SBJ', 'NP_SBJ'], [7, '인공', '인공/NNG', '인공', 'NNG', 'NNG', '_', '_', 8, 8, 'NP', 'NP'], [8, '지능의', '지능/NNG+의/JKG', '지능의', 'NNG+JKG', 'NNG+JKG', '_', '_', 10, 10, 'NP_MOD', 'NP_MOD'], [9, '한', '한/MM', '한', 'MM', 'MM', '_', '_', 10, 10, 'DP', 'DP'], [10, '분야로,', '분야/NNG+로/JKB+,/SP', '분야로,', 'NNG+JKB+SP', 'NNG+JKB+SP', '_', '_', 19, 19, 'NP_AJT', 'NP_AJT'], [11, '기계가', '기